In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
139,application_1580996944851_0095,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


In [4]:
// convert the label from {0, 1} to {1, 2}
//val labelConverter = new FuncTransformer(udf {d: Double => if (d==0) 2 else d }).setInputCol("Class").setOutputCol("Class")
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_efe9741e0182
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_0b100ce905ba
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_7b1483af4614
pipeline: org.apache.spark.ml.Pipeline = pipeline_bf3bdecab0da
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_bf3bdecab0da
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

### step 2. split the dataset into training and validation dataset.

In [5]:
val splitTime = data.stat.approxQuantile("Time", Array(0.7), 0.001).head
    val trainingData = data.filter(s"Time<$splitTime").cache()
    val validationData = data.filter(s"Time>=$splitTime").cache()
    println("Split data into Training and Validation: ")
    println("training records count: " + trainingData.count())
    println("validation records count: " + validationData.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime: Double = 132970.0
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
Split data into Training and Validation: 
training records count: 199440
validation records count: 85367


In [6]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" Validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|199056|
|  1.0|   384|
+-----+------+

 Validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+



In [7]:
val rfModel = {
    val rfGridSearch = for (
    rfNumTrees <- Array(10, 15, 20);
    rfImpurity <- Array("entropy","gini");
    rfMaxBins <- Array(24, 28, 32);
    rfmaxDepth <- Array(4, 6, 8)) 
    yield {
   println(s"Training random forest numTrees : $rfNumTrees, impurity : $rfImpurity, maxBins: $rfMaxBins, maxDepth : $rfmaxDepth")     
   val rfModel = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(rfNumTrees).setImpurity(rfImpurity).setMaxDepth(rfmaxDepth).setSeed(42).setMaxBins(rfMaxBins).fit(trainingData)
   val predictionsRF = rfModel.transform(validationData)      
   val rfAUC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC").evaluate(predictionsRF)  
   println("Area Under ROC Curve = " + rfAUC)
        ((rfNumTrees, rfImpurity, rfMaxBins, rfmaxDepth), rfModel, rfAUC)
    }
    
    println(rfGridSearch.sortBy(-_._3).take(5).mkString("\n"))
        val BestModel = rfGridSearch.sortBy(-_._3).head._2
}   

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training random forest numTrees : 10, impurity : entropy, maxBins: 24, maxDepth : 4
Area Under ROC Curve = 0.8194327154774146
Training random forest numTrees : 10, impurity : entropy, maxBins: 24, maxDepth : 6
Area Under ROC Curve = 0.865723147290196
Training random forest numTrees : 10, impurity : entropy, maxBins: 24, maxDepth : 8
Area Under ROC Curve = 0.865717282806681
Training random forest numTrees : 10, impurity : entropy, maxBins: 28, maxDepth : 4
Area Under ROC Curve = 0.8101793207016702
Training random forest numTrees : 10, impurity : entropy, maxBins: 28, maxDepth : 6
Area Under ROC Curve = 0.851834258401307
Training random forest numTrees : 10, impurity : entropy, maxBins: 28, maxDepth : 8
Area Under ROC Curve = 0.8564638880309366
Training random forest numTrees : 10, impurity : entropy, maxBins: 32, maxDepth : 4
Area Under ROC Curve = 0.8101793207016702
Training random forest numTrees : 10, impurity : entropy, maxBins: 32, maxDepth : 6
Area Under ROC Curve = 0.856463888030

In [8]:
//Gradient-boosted tree classifier
// Train a GBT model.
//setImpurity("entropy") 
val t = System.nanoTime
val gbt = new GBTClassifier().setLabelCol("label").setFeaturesCol("features").setMaxIter(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 6012717937604939
gbt: org.apache.spark.ml.classification.GBTClassifier = gbtc_85c037d429cd


In [9]:
val modelGBT = gbt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelGBT: org.apache.spark.ml.classification.GBTClassificationModel = GBTClassificationModel (uid=gbtc_85c037d429cd) with 10 trees
durationtrain: Double = 15.953715621

initial model training finished.
Training process takes 15.953715621 secs


In [10]:
val s = System.nanoTime
val predictionsGBT = modelGBT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsGBT.cache()
//val predictionsAndLabel: RDD[Row] = df.rdd= predictionsGBT.select("prediction", "label")
predictionsGBT.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 6012736194945244
predictionsGBT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.264481375

initial model training finished.
Training process takes 0.264481375 secs
res21: predictionsGBT.type = [V1: double, V2: double ... 35 more fields]
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [10]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsGBT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85285|
|       1.0|   82|
+----------+-----+



In [11]:
println(s"Matrice de confusion :")
predictionsGBT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85248|
|       1.0|  0.0|   11|
|       0.0|  1.0|   37|
|       1.0|  1.0|   71|
+----------+-----+-----+



In [12]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsGBT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsGBT))
println("Accuracy = " + evaluator1.evaluate(predictionsGBT))
println("Precision = " + evaluator2.evaluate(predictionsGBT))
println("Recall = " + evaluator3.evaluate(predictionsGBT))
println("F1 = " + evaluator4.evaluate(predictionsGBT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_50fcba2ce0b7
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1087a3f176d5
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e804f4e79b36
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_50d8a0d2a048
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_d7131bcadcb0
accuracy: Double = 0.9994377218363067
Area Under ROC Curve = 0.8286391943850394
Accuracy = 0.9994377218363067
Precision = 0.9993969973754311
Recall = 0.9994377218363067
F1 = 0.9993992930330633
Test Error = 5.622781636932528E-4


In [13]:
//val metrics = new BinaryClassificationMetrics(predictionsAndLabel)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
// Random forest classifier
// Train a RandomForest model.
val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_de6ee969327a
t: Long = 629651203469028


In [15]:
val modelRF = rf.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelRF: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_de6ee969327a) with 10 trees
durationtrain: Double = 3.238496364

initial model training finished.
Training process takes 3.238496364 secs


In [16]:
val s = System.nanoTime
val predictionsRF = modelRF.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsRF.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629654954133309
predictionsRF: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.23815253

initial model training finished.
Training process takes 0.23815253 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.84071812067749...|
|       0.0|  0.0|[0.83224682802076...|
|       0.0|  0.0|[0.84713202953547...|
|       0.0|  0.0|[0.81014624583389...|
|       0.0|  0.0|[0.80346968413767...|
+----------+-----+--------------------+
only showing top 5 rows



In [17]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsRF.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85289|
|       1.0|   78|
+----------+-----+



In [18]:
println(s"Matrice de confusion :")
predictionsRF.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85258|
|       1.0|  0.0|    1|
|       0.0|  1.0|   31|
|       1.0|  1.0|   77|
+----------+-----+-----+



In [19]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsRF)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsRF))
println("Accuracy = " + evaluator1.evaluate(predictionsRF))
println("Precision = " + evaluator2.evaluate(predictionsRF))
println("Recall = " + evaluator3.evaluate(predictionsRF))
println("F1 = " + evaluator4.evaluate(predictionsRF))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_83e0648ec089
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_18e451b60167
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6dccc8d16102
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_f9acdddfab63
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_9f0c1ce0279a
accuracy: Double = 0.9996251478908712
Area Under ROC Curve = 0.8564756169979665
Accuracy = 0.9996251478908712
Precision = 0.9996207701889245
Recall = 0.9996251478908712
F1 = 0.9995949508509803
Test Error = 3.748521091287982E-4


In [20]:
// Decision Tree classifier
// Train a DecisionTree model.
val dt = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_b140cd877ae3
t: Long = 629665084108735


In [21]:
val modelDT = dt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelDT: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_b140cd877ae3) of depth 5 with 31 nodes
durationtrain: Double = 2.146376681

initial model training finished.
Training process takes 2.146376681 secs


In [22]:
val s = System.nanoTime
val predictionsDT = modelDT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsDT.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629667975336988
predictionsDT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.227669649

initial model training finished.
Training process takes 0.227669649 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.84071812067749...|
|       0.0|  0.0|[0.83224682802076...|
|       0.0|  0.0|[0.84713202953547...|
|       0.0|  0.0|[0.81014624583389...|
|       0.0|  0.0|[0.80346968413767...|
+----------+-----+--------------------+
only showing top 5 rows



In [23]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsDT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85286|
|       1.0|   81|
+----------+-----+



In [24]:
println(s"Matrice de confusion :")
predictionsDT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85254|
|       1.0|  0.0|    5|
|       0.0|  1.0|   32|
|       1.0|  1.0|   76|
+----------+-----+-----+



In [25]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsDT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsDT))
println("Accuracy = " + evaluator1.evaluate(predictionsDT))
println("Precision = " + evaluator2.evaluate(predictionsDT))
println("Recall = " + evaluator3.evaluate(predictionsDT))
println("F1 = " + evaluator4.evaluate(predictionsDT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_3e4708f1a14e
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_54388aedc73e
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_2b4679a688cd
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5787a212a6d1
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_b55bae4f9286
accuracy: Double = 0.9995665772488198
Area Under ROC Curve = 0.851822529434277
Accuracy = 0.9995665772488198
Precision = 0.999547172372845
Recall = 0.9995665772488198
F1 = 0.9995356527897536
Test Error = 4.3342275118019025E-4


In [26]:
/* val trainingData1 = trainingData.withColumn("label", when(col("label") === 0, -1)
                                   .otherwise(col("label"))
                           );
val validationData1 = validationData.withColumn("label", when(col("label") === 0, -1)
                                   .otherwise(col("label"))
                           );*/

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
// SVM classifier
// Train a SVM model.
val svm = new LinearSVC().setLabelCol("label").setFeaturesCol("features").setMaxIter(100).setRegParam(0.1) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

svm: org.apache.spark.ml.classification.LinearSVC = linearsvc_d3980dafb2e7
t: Long = 629676769761255


In [28]:
val modelSVM = svm.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelSVM: org.apache.spark.ml.classification.LinearSVCModel = linearsvc_d3980dafb2e7
durationtrain: Double = 61.191874208

initial model training finished.
Training process takes 61.191874208 secs


In [29]:
val s = System.nanoTime
val predictionsSVM = modelSVM.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsSVM.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629740594975966
predictionsSVM: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.209021799

initial model training finished.
Training process takes 0.209021799 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.84071812067749...|
|       0.0|  0.0|[0.83224682802076...|
|       0.0|  0.0|[0.84713202953547...|
|       0.0|  0.0|[0.81014624583389...|
|       0.0|  0.0|[0.80346968413767...|
+----------+-----+--------------------+
only showing top 5 rows



In [30]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsSVM.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85367|
+----------+-----+



In [31]:
println(s"Matrice de confusion :")
predictionsSVM.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85259|
|       0.0|  1.0|  108|
+----------+-----+-----+



In [32]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsSVM)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsSVM))
println("Accuracy = " + evaluator1.evaluate(predictionsSVM))
println("Precision = " + evaluator2.evaluate(predictionsSVM))
println("Recall = " + evaluator3.evaluate(predictionsSVM))
println("F1 = " + evaluator4.evaluate(predictionsSVM))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_816cee599cf2
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_59cf454254df
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_7488c30cc813
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_be1855853843
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4384a712406c
accuracy: Double = 0.9987348741316903
Area Under ROC Curve = 0.5
Accuracy = 0.9987348741316903
Precision = 0.9974713488068432
Recall = 0.9987348741316903
F1 = 0.9981027115866723
Test Error = 0.0012651258683097355


In [33]:
// Logistic Regression classifier
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true).setRegParam(0.3).setElasticNetParam(0.8)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_0bcb939909b1
t: Long = 629748255603035


In [34]:
// train the multiclass model.
//val Modelovr = ovr.fit(trainingData)
val Modellr = lr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Modellr: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_0bcb939909b1, numClasses = 2, numFeatures = 11
durationtrain: Double = 1.465245704

initial model training finished.
Training process takes 1.465245704 secs


In [35]:
val s = System.nanoTime
val predictionsLR = Modellr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsLR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629750147059268
predictionsLR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.200476376

initial model training finished.
Training process takes 0.200476376 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.84071812067749...|
|       0.0|  0.0|[0.83224682802076...|
|       0.0|  0.0|[0.84713202953547...|
|       0.0|  0.0|[0.81014624583389...|
|       0.0|  0.0|[0.80346968413767...|
+----------+-----+--------------------+
only showing top 5 rows



In [36]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsLR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85367|
+----------+-----+



In [37]:
println(s"Matrice de confusion :")
predictionsLR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85259|
|       0.0|  1.0|  108|
+----------+-----+-----+



In [38]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsLR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsLR))
println("Accuracy = " + evaluator1.evaluate(predictionsLR))
println("Precision = " + evaluator2.evaluate(predictionsLR))
println("Recall = " + evaluator3.evaluate(predictionsLR))
println("F1 = " + evaluator4.evaluate(predictionsLR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_fcb51b0447a2
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_cda682de502a
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_361d3df26b97
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_da36650086fd
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_60aec12f69ed
accuracy: Double = 0.9987348741316903
Area Under ROC Curve = 0.5
Accuracy = 0.9987348741316903
Precision = 0.9974713488068432
Recall = 0.9987348741316903
F1 = 0.9981027115866723
Test Error = 0.0012651258683097355


In [39]:
// Naive Bayes classifier
// Train a NaiveBayes model.
val nb = new NaiveBayes().setLabelCol("label").setFeaturesCol("features").setSmoothing(1.0)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nb: org.apache.spark.ml.classification.NaiveBayes = nb_5613a1322861
t: Long = 629757756504823


In [40]:
/*val scaler = new MinMaxScaler().setInputCol("features").setOutputCol("featuresScaled")
val pipeline = new Pipeline().setStages(Array(scaler))
val pipelineModel1 = pipeline.fit(trainingData)
val pipelineModel2 = pipeline.fit(trainingData)
val datatrain = pipelineModel1.transform(trainingData)
val datatest = pipelineModel2.transform(validationData)*/

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
val modelNB = nb.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelNB: org.apache.spark.ml.classification.NaiveBayesModel = NaiveBayesModel (uid=nb_5613a1322861) with 2 classes
durationtrain: Double = 1.381058102

initial model training finished.
Training process takes 1.381058102 secs


In [42]:
val s = System.nanoTime
val predictionsNB = modelNB.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsNB.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629759887515209
predictionsNB: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.212813275

initial model training finished.
Training process takes 0.212813275 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [43]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsNB.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85367|
+----------+-----+



In [44]:
println(s"Matrice de confusion :")
predictionsNB.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85259|
|       0.0|  1.0|  108|
+----------+-----+-----+



In [45]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsNB)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsNB))
println("Accuracy = " + evaluator1.evaluate(predictionsNB))
println("Precision = " + evaluator2.evaluate(predictionsNB))
println("Recall = " + evaluator3.evaluate(predictionsNB))
println("F1 = " + evaluator4.evaluate(predictionsNB))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9c4f9da90fde
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e571dbdcc56d
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5551412f2bef
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_fc1168ed2cf6
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_efa0db166513
accuracy: Double = 0.9987348741316903
Area Under ROC Curve = 0.5
Accuracy = 0.9987348741316903
Precision = 0.9974713488068432
Recall = 0.9987348741316903
F1 = 0.9981027115866723
Test Error = 0.0012651258683097355


In [46]:
// Multilayer Perceptron Classifier
// create the trainer and set its parameters
val t = System.nanoTime
val layers = Array[Int] (11, 6, 3, 2)
val mlp = new MultilayerPerceptronClassifier().setLayers(layers).setLabelCol("label").setFeaturesCol("features").setTol(1E-4).setBlockSize(128).setSeed(1234L).setMaxIter(25) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 629767957201401
layers: Array[Int] = Array(11, 6, 3, 2)
mlp: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_ea0a4bdb9d92


In [47]:
// Train a Multilayer Perceptron model.
val modelMLP = mlp.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelMLP: org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel = mlpc_ea0a4bdb9d92
durationtrain: Double = 8.1446229

initial model training finished.
Training process takes 8.1446229 secs


In [48]:
val s = System.nanoTime
val predictionsMLP = modelMLP.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsMLP.select("prediction","label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629778044698795
predictionsMLP: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.204806983

initial model training finished.
Training process takes 0.204806983 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [49]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsMLP.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85272|
|       1.0|   95|
+----------+-----+



In [50]:
println(s"Matrice de confusion :")
predictionsMLP.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85239|
|       1.0|  0.0|   20|
|       0.0|  1.0|   33|
|       1.0|  1.0|   75|
+----------+-----+-----+



In [51]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsMLP)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsMLP))
println("Accuracy = " + evaluator1.evaluate(predictionsMLP))
println("Precision = " + evaluator2.evaluate(predictionsMLP))
println("Recall = " + evaluator3.evaluate(predictionsMLP))
println("F1 = " + evaluator4.evaluate(predictionsMLP))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9a7804cf607b
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_00eec797f0db
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9248b0207d11
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8a46816aa8a2
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_0184048cc883
accuracy: Double = 0.9993791511942554
Area Under ROC Curve = 0.8471049325519234
Accuracy = 0.9993791511942554
Precision = 0.9993471504077043
Recall = 0.9993791511942554
F1 = 0.9993592954634178
Test Error = 6.208488057446448E-4


In [52]:
// One Vs Rest Classifier using Logistic Regression classifier
val classifier = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

classifier: org.apache.spark.ml.classification.LogisticRegression = logreg_012c12484a81
t: Long = 629786630362449


In [53]:
// train the multiclass model.
val ovr = new OneVsRest().setClassifier(classifier)
val Modelovr = ovr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ovr: org.apache.spark.ml.classification.OneVsRest = oneVsRest_8cd085ec9f37
Modelovr: org.apache.spark.ml.classification.OneVsRestModel = oneVsRest_8cd085ec9f37
durationtrain: Double = 4.573582428

initial model training finished.
Training process takes 4.573582428 secs


In [54]:
val s = System.nanoTime
val predictionsOVR = Modelovr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsOVR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 629792600616211
predictionsOVR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.423795385

initial model training finished.
Training process takes 0.423795385 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.84071812067749...|
|       0.0|  0.0|[0.83224682802076...|
|       0.0|  0.0|[0.84713202953547...|
|       0.0|  0.0|[0.81014624583389...|
|       0.0|  0.0|[0.80346968413767...|
+----------+-----+--------------------+
only showing top 5 rows



In [55]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsOVR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85259|
|  1.0|  108|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85334|
|       1.0|   33|
+----------+-----+



In [56]:
println(s"Matrice de confusion :")
predictionsOVR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85259|
|       0.0|  1.0|   75|
|       1.0|  1.0|   33|
+----------+-----+-----+



In [57]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val areaUnderPR = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderPR")
val accuracy = evaluator1.evaluate(predictionsOVR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsOVR))
println("Area Under the Precision-Recall Curve = "  + areaUnderPR.evaluate(predictionsOVR))
println("Accuracy = " + evaluator1.evaluate(predictionsOVR))
println("Precision = " + evaluator2.evaluate(predictionsOVR))
println("Recall = " + evaluator3.evaluate(predictionsOVR))
println("F1 = " + evaluator4.evaluate(predictionsOVR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e51eebbc1b66
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_88ff04c328dd
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6c05c29c158b
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5a178875b67d
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_ee7f9fe8f129
areaUnderPR: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_742162e3e9ff
accuracy: Double = 0.9991214403692293
Area Under ROC Curve = 0.6527777777777778
Area Under the Precision-Recall Curve = 0.6532170575931632
Accuracy = 0.9991214403692293
Precision = 0.9991222125347473
Recall = 0.9991214403692293
F1 = 0.9988879740192093
Test Error = 8.785596307706589E-4
